In [1]:
#lancer spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1701655161658_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Librairies pour la manipulation de données
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf
from pyspark.sql.types import *

# Librairies pour le traitement d'images
from PIL import Image
import numpy as np
import io
import os
from keras_preprocessing.image import img_to_array
from pyspark.ml.image import ImageSchema

# Librairies pour la visualisation
import matplotlib.pyplot as plt

# Librairies pour l'apprentissage automatique (Deep Learning)
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# Librairies pour le traitement des données vectorielles
from pyspark.ml.feature import PCA  
from pyspark.ml.linalg import Vectors, VectorUDT

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Chemin vers le répertoire des données
PATH_Data = 's3://buckets-data/Test'

# Chemin vers le répertoire des résultats
PATH_Result = 's3://buckets-data/Results'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Lecture des fichiers binaires (images) à partir d'un répertoire avec Spark
images = (
    spark  # Utilisation de la session Spark précédemment créée
    .read  # Méthode pour lire des données dans Spark
    .format("binaryFile")  # Spécification du format des fichiers (ici, fichiers binaires)
    .option("pathGlobFilter", "*.jpg")  # Filtrage des fichiers pour inclure uniquement ceux avec l'extension .jpg
    .option("recursiveFileLookup", "true")  # Activation de la recherche récursive des fichiers dans les sous-répertoires
    .load(PATH_Data)  # Chargement des fichiers binaires à partir du chemin spécifié (PATH_Data)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Visualisation
images.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://buckets-data...|2023-12-01 22:27:14| 78796|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 78179|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 78142|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 78115|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 77820|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 77580|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 77527|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:13| 77515|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 77506|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 77421|[FF D8 FF E0 00 1...|
|s3://buckets-data...|2023-12-01 22:27:14| 77205|[FF D8 FF E0 00 1...|
|s3://

In [7]:
# Ajout d'une colonne 'label' au DataFrame 'images'
images = images.withColumn('label', element_at(split(images['path'], '/'), -2))

# Impression du schéma du DataFrame 'images'
print(images.printSchema())

# Sélection des colonnes 'path' et 'label' et affichage des six premières lignes
print(images.select('path', 'label').show(6, False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+----------------------------------------+------+
|path                                    |label |
+----------------------------------------+------+
|s3://buckets-data/Test/pear_3/r0_148.jpg|pear_3|
|s3://buckets-data/Test/pear_3/r0_146.jpg|pear_3|
|s3://buckets-data/Test/pear_3/r0_142.jpg|pear_3|
|s3://buckets-data/Test/pear_3/r0_132.jpg|pear_3|
|s3://buckets-data/Test/pear_3/r0_134.jpg|pear_3|
|s3://buckets-data/Test/pear_3/r0_194.jpg|pear_3|
+----------------------------------------+------+
only showing top 6 rows

None

In [8]:
# Chargement du modèle MobileNetV2 pré-entraîné sur ImageNet
# Le paramètre 'weights' spécifie que les poids pré-entraînés sur le jeu de données ImageNet doivent être utilisés.
# Le paramètre 'include_top' indique si les couches fully connected (FC) situées au sommet du réseau doivent être incluses.
# Le paramètre 'input_shape' définit la forme attendue de l'entrée du modèle.
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

# Création d'un nouveau modèle en excluant la dernière couche fully connected (FC)
# Cela permet d'obtenir un modèle sans la couche de prédiction d'ImageNet.
new_model = Model(inputs=model.input,           
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

In [9]:
# Affichage du résumé du nouveau modèle 
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [10]:
broadcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def model_fn():
    """
    Fonction pour créer un nouveau modèle basé sur MobileNetV2 avec des poids pré-entrainés.

    Returns:
    - new_model (keras.Model): Un modèle basé sur MobileNetV2 avec les poids pré-entrainés.
    """

    # Chargement du modèle MobileNetV2 pré-entrainé sur ImageNet. Le modèle complet est chargé avec toutes ses couches.
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))

    # Désactivation de l'entraînement pour toutes les couches du modèle. Ceci est fait pour geler les poids du modèle pré-entrainé afin de les conserver.
    for layer in model.layers:
        layer.trainable = False

    # Création d'un nouveau modèle en excluant la dernière couche fully connected (FC). Cela permet d'obtenir un modèle sans la couche de prédiction d'ImageNet.
    new_model = Model(inputs=model.input,
                      outputs=model.layers[-2].output)

    # Initialisation des poids du nouveau modèle avec les poids diffusés
    new_model.set_weights(broadcast_weights.value)

    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def preprocess(content):
    """
    Fonction de prétraitement pour convertir le contenu d'une image en tableau d'entrée pour le modèle.

    Args:
    - content (bytes): Contenu de l'image en format binaire.

    Returns:
    - arr (numpy.ndarray): Tableau d'entrée prétraité pour le modèle.
    """
    # Ouverture de l'image, redimensionnement et conversion en tableau
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    # Prétraitement spécifique à MobileNetV2
    return preprocess_input(arr)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def featurize_series(model, content_series):
    """
    Fonction pour extraire les caractéristiques (features) à partir d'une série de contenus d'images.

    Args:
    - model (keras.Model): Modèle utilisé pour l'extraction des caractéristiques.
    - content_series (pandas.Series): Série de contenus d'images.

    Returns:
    - pd.Series: Série de caractéristiques extraites.
    """
    # Prétraitement des images dans la série
    input = np.stack(content_series.map(preprocess))
    # Prédiction des caractéristiques à l'aide du modèle
    preds = model.predict(input)
    # Aplatir les caractéristiques pour un stockage plus facile dans les DataFrames Spark
    output = [p.flatten() for p in preds]
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    """
    Fonction UDF Pandas pour extraire les caractéristiques d'un lot de contenus d'images.

    Args:
    - content_series_iter: Itérateur de séries de contenus d'images.

    Yields:
    - pd.Series: Série de caractéristiques extraites.
    """
    # Chargement du modèle une seule fois pour amortir les frais généraux
    model = model_fn()
    for content_series in content_series_iter:
        # Rendement des caractéristiques extraites pour chaque série de contenus
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1701655161658_0001/container_1701655161658_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:403: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [15]:
# Limitation du nombre de lignes par lot lors de l'utilisation d'Arrow
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Création d'un DataFrame de caractéristiques à partir des images
features_df = (
    images  # DataFrame contenant les informations sur les images
    .repartition(20)  # Répartition des données en 20 partitions pour le traitement parallèle
    .select(
        col("path"),  # Sélection de la colonne 'path' contenant les chemins des images
        col("label"),  # Sélection de la colonne 'label' contenant les libellés des images
        featurize_udf("content").alias("features")  # Application de la fonction UDF pour extraire les caractéristiques
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                path|               label|            features|
+--------------------+--------------------+--------------------+
|s3://buckets-data...|apple_red_delicios_1|[0.09279204, 0.0,...|
|s3://buckets-data...|              pear_3|[0.50657016, 0.08...|
|s3://buckets-data...|apple_red_delicios_1|[0.8578642, 0.306...|
|s3://buckets-data...|apple_red_delicios_1|[0.52145797, 0.04...|
|s3://buckets-data...|apple_red_delicios_1|[0.5543406, 0.176...|
|s3://buckets-data...|apple_red_delicios_1|[0.48165464, 0.03...|
|s3://buckets-data...|      apple_golden_1|[0.0, 0.5390478, ...|
|s3://buckets-data...|      apple_golden_1|[0.0, 0.15885179,...|
|s3://buckets-data...|      apple_golden_1|[0.0, 0.41714785,...|
|s3://buckets-data...|   apple_pink_lady_1|[0.5494931, 0.0, ...|
+--------------------+--------------------+--------------------+
only showing top 10 rows

In [18]:
#Construction d'une User defined Fonction qui transforme les listes de features en vecteurs denses.
array_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Création d'un DataFrame vectorisé à partir des caractéristiques extraites
vectorized_df = features_df.withColumn('cnn_vectors', array_to_vector_udf('features'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
vectorized_df.show(6, True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+
|                path|               label|            features|         cnn_vectors|
+--------------------+--------------------+--------------------+--------------------+
|s3://buckets-data...|              pear_3|[1.129328, 0.2556...|[1.12932801246643...|
|s3://buckets-data...|              pear_3|[0.6876476, 0.224...|[0.68764758110046...|
|s3://buckets-data...|              pear_3|[0.103774644, 0.0...|[0.10377464443445...|
|s3://buckets-data...|              pear_3|[0.44024852, 0.44...|[0.44024851918220...|
|s3://buckets-data...|apple_red_delicios_1|[0.46985993, 0.17...|[0.46985992789268...|
|s3://buckets-data...|apple_red_delicios_1|[0.24741146, 0.07...|[0.24741145968437...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 6 rows

In [21]:
# Fonction pour réduction de dimensions PCA
def reduc_dim_pca(features):
    '''Réduction des dimensions PCA sur les feature
    Entrée:
    - features de l\'image
    
    Retour:
    - pca
    - pca_matrix: résultat de la réduction de dimensions
    '''
    # Conversion d'un tableau (array) en vecteur dense
    # Création d'un fonction utilisateur avec udf
    #vector_dense = udf(lambda x: Vectors.dense(x), VectorUDT())
    img_vd_df = features.select("path","label","cnn_vectors")
    
    # PCA
    pca_spark = PCA(inputCol="cnn_vectors", outputCol="features_pca", k=100)
    pca = pca_spark.fit(img_vd_df)
    pca_matrix = pca.transform(img_vd_df)
    
    return pca, pca_matrix

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Appelle de la fonction pour la réduction de dimension PCA
pca, pca_matrix = reduc_dim_pca(vectorized_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
pca_matrix.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+
|                path|               label|         cnn_vectors|        features_pca|
+--------------------+--------------------+--------------------+--------------------+
|s3://buckets-data...|              pear_3|[1.12932801246643...|[5.17719494077324...|
|s3://buckets-data...|              pear_3|[0.68764758110046...|[5.72051562223153...|
|s3://buckets-data...|              pear_3|[0.10377464443445...|[0.03909529548625...|
|s3://buckets-data...|              pear_3|[0.44024851918220...|[-0.1034632370571...|
|s3://buckets-data...|apple_red_delicios_1|[0.09279204159975...|[-6.4526393018356...|
|s3://buckets-data...|              pear_3|[0.50657016038894...|[3.06740773501763...|
|s3://buckets-data...|apple_red_delicios_1|[0.46985992789268...|[-8.6126005945394...|
|s3://buckets-data...|apple_red_delicios_1|[0.24741145968437...|[-10.288431641490...|
|s3://buckets-data...|apple_red_delicios_1|[0.00111465

In [24]:
# Fonction pour l'affichage du diagramme des éboulis PCA
def display_pca_eboulis(pca):
    '''Affichage du diagramme des éboulis sur PCA
    Entrée:
    - pca
    
    Retour:
    - N/A
    '''
    # Variance expliquée
    varexpl = pca.explainedVariance*100
        
    # Affichage de la variance cumulée
    plt.figure(figsize=(11,7))
    plt.bar(np.arange(len(varexpl))+1, varexpl)

    cumSumVar = varexpl.cumsum()
    plt.plot(np.arange(len(varexpl))+1, cumSumVar,c="red",marker='o')
    plt.axhline(y=90, linestyle="--", 
                color="green",
                linewidth=1)

    limit = 90
    valid_idx = np.where(cumSumVar >= limit)[0]
    min_plans = valid_idx[cumSumVar[valid_idx].argmin()]+1
    plt.axvline(x=min_plans, linestyle="--", 
                color="green",
                linewidth=1)

    plt.xlabel("Rang de l'axe d'inertie pour les composantes K")
    plt.xticks(np.arange(0,105,5))
    plt.xlim(left=0)
    plt.ylabel("Pourcentage d'inertie")
    plt.title("{}% de la variance totale est expliquée"\
                " par les {} premiers axes".format(limit,
                                            min_plans))
    plt.show(block=False)
        
    return None

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Affichage du diagramme des éboulis
display_pca_eboulis(pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Transformation inverse (de Vectors à Arrays)
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
final_df = pca_matrix.withColumn('features', vector_to_array_udf('features_pca'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
final_df.show(6, True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                path|               label|         cnn_vectors|        features_pca|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|s3://buckets-data...|              pear_3|[0.50564825534820...|[3.87160660262834...|[3.8716066, 8.488...|
|s3://buckets-data...|              pear_3|[0.59719002246856...|[1.16041364501100...|[1.1604136, 8.267...|
|s3://buckets-data...|apple_red_delicios_1|[0.85786420106887...|[-13.991283388120...|[-13.991283, 1.48...|
|s3://buckets-data...|apple_red_delicios_1|[0.46985992789268...|[-8.6126005945394...|[-8.6126, 1.11063...|
|s3://buckets-data...|apple_red_delicios_1|[0.24741145968437...|[-10.288431641490...|[-10.288431, -1.4...|
|s3://buckets-data...|apple_red_delicios_1|[0.00111465062946...|[-8.5432868782433...|[-8.543287, 0.347...|
+--------------------+---------------

In [29]:
# Enregistrement des données (format parquet)
final_df.write.mode('overwrite').parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                   path  ...                                           features
0                s3://buckets-data/Test/pear_3/r0_4.jpg  ...  [5.177195, 7.8471303, 1.692013, 19.994278, 1.2...
1                s3://buckets-data/Test/pear_3/r0_8.jpg  ...  [5.7205157, 6.9522395, 2.2764564, 17.884733, 2...
2               s3://buckets-data/Test/pear_3/r0_60.jpg  ...  [0.039095294, 7.7602572, 3.8032453, 10.0871935...
3              s3://buckets-data/Test/pear_3/r0_270.jpg  ...  [-0.10346324, 9.0959215, 3.2954853, 13.42263, ...
4     s3://buckets-data/Test/apple_red_delicios_1/r1...  ...  [-6.452639, -0.09719531, 2.3864064, 2.2326634,...
...                                                 ...  ...                                                ...
1375           s3://buckets-data/Test/apple_6/r1_28.jpg  ...  [-0.44116402, 13.907021, -3.1299317, 4.796015,...
1376          s3://buckets-data/Test/apple_6/r0_200.jpg  ...  [-1.5895584, 15.993085, -6.1968613, 4.4463